In [1]:
import gymnasium as gym
from stable_baselines3 import PPO
from stable_baselines3.common.vec_env import DummyVecEnv
import numpy as np

/opt/anaconda3/envs/test/lib/python3.11/site-packages/gym/envs/registration.py:307: DeprecationWarning: The package name gym_minigrid has been deprecated in favor of minigrid. Please uninstall gym_minigrid and install minigrid with `pip install minigrid`. Future releases will be maintained under the new package name minigrid.
  fn()


In [2]:
env_name = "BipedalWalker-v3"
env = gym.make(env_name)

In [3]:
# 환경을 벡터라이즈(멀티 환경 지원)하기 위해 DummyVecEnv 사용
# (Stable-Baselines3에서 학습하려면 벡터라이즈된 환경이 필요)
vec_env = DummyVecEnv([lambda: gym.make(env_name)])

# 모델 생성 (정책 네트워크: MlpPolicy)
# - 여기에 다양한 하이퍼파라미터를 조정할 수 있습니다.
model = PPO(
    policy="MlpPolicy",
    env=vec_env,
    verbose=1,          # 학습 정보를 출력하도록 설정
    n_steps=2048,       # 타임스텝마다 업데이트할 샘플 수
    batch_size=64,
    gae_lambda=0.95,
    gamma=0.99,
    n_epochs=10,
    learning_rate=5e-4,
    clip_range=0.3
)

# 학습 진행
total_timesteps = 300_000  # 필요에 따라 증가/감소
model.learn(total_timesteps=total_timesteps)

# 모델 저장
model.save("ppo_bipedalwalker")
print("학습된 모델을 저장했습니다.")

Using cpu device
-----------------------------
| time/              |      |
|    fps             | 6303 |
|    iterations      | 1    |
|    time_elapsed    | 0    |
|    total_timesteps | 2048 |
-----------------------------
----------------------------------------
| time/                   |            |
|    fps                  | 4731       |
|    iterations           | 2          |
|    time_elapsed         | 0          |
|    total_timesteps      | 4096       |
| train/                  |            |
|    approx_kl            | 0.02172231 |
|    clip_fraction        | 0.0784     |
|    clip_range           | 0.3        |
|    entropy_loss         | -5.65      |
|    explained_variance   | 0.0186     |
|    learning_rate        | 0.0005     |
|    loss                 | 1.46       |
|    n_updates            | 10         |
|    policy_gradient_loss | -0.00867   |
|    std                  | 0.991      |
|    value_loss           | 39.1       |
-----------------------------------

In [ ]:
# 학습된 모델로 테스트
test_env = gym.make(env_name, render_mode="human")
obs, _ = test_env.reset()
model = PPO.load("ppo_bipedalwalker", env=test_env)
done = False
total_reward = 0.0

while not done:
    # 모델이 관측(obs)을 바탕으로 액션을 결정
    action, _ = model.predict(obs, deterministic=True)
    obs, reward, terminated, truncated, _ = test_env.step(action)
    done = terminated or truncated
    total_reward += reward

test_env.close()
print(f"테스트 에피소드 리워드: {total_reward:.2f}")

Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
테스트 에피소드 리워드: 258.39


: 